# Order Book

In [39]:
from cbpro.public_client import PublicClient
from typing import Dict, List, Tuple
import numpy as np

In [2]:
client = PublicClient()
ob_all = client.get_product_order_book('ETH-GBP', 3)

{'sequence': 110415955, 'bids': [['103.56', '0.59832687', '01bc3ff0-8f70-4dd7-8dc7-cbc64f499303'], ['103.55', '7.199', '13b5f25f-b644-4da3-94d4-b80d9ff95b07'], ['103.55', '10.92365427', '13e6d547-3cce-483f-bb49-7ea155420d4d'], ['103.54', '4.8', '066f7803-0e2a-47f8-ba33-983fd1a4e2fb'], ['103.53', '11', '86874952-7fe5-4a32-a3a7-70ffc4c40f62'], ['103.45', '0.5', 'd41b2f32-4d62-4b55-b201-e483300ef0ff'], ['103.37', '200.999', '24952255-c6e6-41a3-84db-d35ca9bd9b6d'], ['103.33', '1', 'd43672ba-ba77-46f4-8a2c-2442df2a4a1f'], ['103.26', '3', 'b323e958-6059-4f34-8cc6-3643efe0492e'], ['103.15', '30.781', '6e15f00d-ee84-4adf-aea0-f8a6bec06531'], ['103.08', '110', '77e3d594-8d44-40c0-958b-5200e3c12dc9'], ['103.07', '1', '732b82e5-2daf-414c-a1c2-d00f4acbaceb'], ['103.03', '48.344', 'fca30f75-2249-4ede-aece-8e68dd9e01fa'], ['102.6', '0.34829434', '474de8fa-7917-4f4b-8070-6b69fe5395fd'], ['102.5', '0.1', '2ba53021-8f7a-4eca-8f61-60c33fe29871'], ['102.5', '8', 'b4af9460-c8d7-4bd2-8183-a5cf6abc07b0'], [

In [3]:
ob_all.keys()

dict_keys(['sequence', 'bids', 'asks'])

In [6]:
bids = ob_all['bids']

In [7]:
asks = ob_all['asks']

In [65]:
# if bid volume > ask volume you will have r > 0
# if bid volume == ask volume you will have r = 0
# if bid volume < ask volume you will have r < 0
# if bid volume >> ask volume you will have r = 1
# if bid volume << ask volume you will have r = -1
r = lambda vb, va: (vb-va)/(vb+va)

mid_price = lambda bb, ba: (bb+ba)/2.0

def vwap(ob: np.ndarray):
    assert ob.shape[1] == 2, "You ob should only contain prices and respective volumes."
    return ob[:, 0] @ ob[:, 1] / np.sum(ob[:, 1])

In [52]:
def aggregate_ob(num_top_orders: int, ob: Dict[str, List[List[str]]]) -> Tuple[np.ndarray, np.ndarray]:
    """
    Dict val is: ['px', 'volume', 'order_id']
    """
    assert all(key_ in ob.keys() for key_ in ['bids', 'asks']), 'No bids or asks in the ob.'
    assert all(len(ob[key_]) >= num_top_orders for key_ in ['bids', 'asks']), 'Not enough orders in either bids or asks.'

    top_bids, top_asks = np.array(ob['bids'][:num_top_orders]), np.array(ob['asks'][:num_top_orders])
    # getting rid of the trade ids
    top_bids, top_asks = top_bids[:, :-1].astype(np.float32), top_asks[:, :-1].astype(np.float32)
    
    return top_bids, top_asks

In [67]:
bids, asks = aggregate_ob(60, ob_all)

In [71]:
vwap(bids), vwap(asks)

(101.904625, 107.30319)

In [72]:
vb, va = np.sum(bids[:, 1]), np.sum(asks[:, 1])

In [76]:
r(vb, va) # i.e. bid volume is slightly larger than the ask volume

0.017156992

In [77]:
mid_price(bids[0][0], asks[0][0])

103.94499969482422